---
title: "Lab 5"
webr:  
  show-startup-message: true    # Display status of webR initialization
  packages: ['data.table', 'tidyverse', 'ggplot2', 'speedglm'] # Pre-install dependency
editor_options: 
  chunk_output_type: console
---

Lab 5 covers 

* **Propensity score analysis** 

```{webr}
library(tidyverse)
diabetes <- read.csv('https://raw.githubusercontent.com/AkiShiroshita/EPID813-2025--Lab/refs/heads/main/data/diabetes.csv')

diabetes_clean <- diabetes %>%
  # Ensure correct variable types
  mutate_if(is.integer, as.numeric) %>%
  # Create exposure and outcome variables
  mutate(
    diab = factor(if_else(glyhb >= 6.5, 1, 0), labels = c("No Diabetes", "Diabetes")),
    htn = factor(if_else(bp.1s > 130, 1, 0), labels = c("No HTN", "HTN"))
  ) %>%
  # Select variables for analysis
  dplyr::select(htn, diab, age, height, weight, frame, hdl) %>%
  # Remove rows with any missing data
  drop_na()
```

* Very simplified workflow

1. Estimate the propensity score

$Pr(Tr=1|Covariates)$

```{webr}
# Model the propensity score
ps_model <- glm(diab ~ age + height + weight + hdl,
                data = diabetes_clean, family = binomial)

# Add the propensity score to our dataframe
diabetes_ps <- diabetes_clean %>%
  mutate(ps_score = predict(ps_model, type = "response"))
```

2. Use the propensity score: weighting, matching, stratification, etc.

Inverse probability weighting (ATE)

* Among treated: $W = 1/PS$

* Among untreated: $W = 1/(1-PS)$

```{webr}
diabetes_ps <- diabetes_ps %>%
  mutate(ipw = if_else(diab == "Diabetes", 1/ps_score, 1/(1 - ps_score)))
```

3. Balance check

Love plot, histogram, etc.

Ensure that the weighted standardized mean difference for each covariate is less than 0.1 (or, if acceptable, less than 0.25).

* Unweighted samples

Continuous variables: $d=100*\frac{\bar{x}_{treatment}-\bar{x}_{control}}{\sqrt{\frac{S^2_{treatment}+S^2_{control}}{2}}}$

Binary variables: $d=100*\frac{\hat{p}_{treatment}-\hat{p}_{control}}{\sqrt{\frac{\hat{p}_{treatment}(1-\hat{p}_{treatment})+\hat{p}_{control}(1-\hat{p}_{control})}{2}}}$

* Weighted samples

$\bar{x}_{weight}=\frac{\sum w_i x_i}{\sum w_i}$

$s^2_{weight}=\frac{\sum w_i}{(\sum w_i)^2-\sum w_i^2} \sum w_i (x_i-\bar{x}_{weight})^2$

```{webr}
weighted_var <- function(x, w, na.rm = TRUE) {
  sum_w <- sum(w)
  sum_w2 <- sum(w^2)
  mean_w <- sum(x * w) / sum_w
  var_w <- sum(w * (x - mean_w)^2) / (sum_w - (sum_w2 / sum_w))
  return(var_w)
} 

calculate_smd <- function(data, treatment_var, covariates, weights_var = NULL) {
  is_weighted <- !is.null(weights_var)
  data$..treatment.. <- factor(data[[treatment_var]])
  
  treat_level <- levels(data$..treatment..)[2]
  control_level <- levels(data$..treatment..)[1]
  
  treat_data <- data[data$..treatment.. == treat_level, ]
  control_data <- data[data$..treatment.. == control_level, ]
  
  smd_list <- list()
  for (cov in covariates) {
    var_t <- treat_data[[cov]]; var_c <- control_data[[cov]]
    w_t <- if (is_weighted) treat_data[[weights_var]] else rep(1, nrow(treat_data))
    w_c <- if (is_weighted) control_data[[weights_var]] else rep(1, nrow(control_data))
    is_binary <- length(unique(na.omit(data[[cov]]))) == 2
    
    if (is_binary) {
      mean_t <- weighted.mean(var_t, w_t, na.rm = TRUE); mean_c <- weighted.mean(var_c, w_c, na.rm = TRUE)
      var_t_stat <- mean_t * (1 - mean_t); var_c_stat <- mean_c * (1 - mean_c)
    } else {
      mean_t <- weighted.mean(var_t, w_t, na.rm = TRUE); mean_c <- weighted.mean(var_c, w_c, na.rm = TRUE)
      var_t_stat <- if (is_weighted) weighted_var(var_t, w_t) else var(var_t, na.rm = TRUE)
      var_c_stat <- if (is_weighted) weighted_var(var_c, w_c) else var(var_c, na.rm = TRUE)
    }
    
    mean_diff <- mean_t - mean_c
    pooled_sd <- sqrt((var_t_stat + var_c_stat) / 2)
    smd <- if (pooled_sd > 0) abs(mean_diff / pooled_sd) else 0
    smd_list[[cov]] <- smd
  }
  
  output_df <- data.frame(Covariate = names(smd_list), SMD = as.numeric(smd_list), row.names = NULL)
  return(output_df)
}

covariates <- c("age", "height", "weight", "hdl")
treatment_var <- "diab"

# 1. Calculate Unweighted SMDs
unweighted_smds <- calculate_smd(
  data = diabetes_ps,
  treatment_var = treatment_var,
  covariates = covariates
) %>%
  dplyr::mutate(Type = "Unweighted")

# 2. Calculate Weighted SMDs
weighted_smds <- calculate_smd(
  data = diabetes_ps,
  treatment_var = treatment_var,
  covariates = covariates,
  weights_var = "ipw" 
) %>%
  dplyr::mutate(Type = "Weighted")

love_data <- dplyr::bind_rows(unweighted_smds, weighted_smds)
love_data$Type <- factor(love_data$Type, levels = c("Unweighted", "Weighted"))

ggplot(data = love_data, aes(x = SMD, y = Covariate, group = Covariate)) +
  geom_point(aes(color = Type), size = 3) +
  geom_vline(xintercept = 0.1, linetype = "dashed", color = "red") +
  labs(
    title = "Covariate Balance Before and After Weighting",
    x = "Absolute Standardized Mean Difference",
    y = "Covariate",
    color = "Sample"
  ) +
  scale_color_manual(values = c("Unweighted" = "grey", "Weighted" = "navy")) +
  theme_minimal(base_size = 14) +
  theme(legend.position = "top")
```


4. Run the outcome analysis

```{webr}
# Outcome analysis
fit <- glm(htn ~ diab, data = diabetes_ps, weights = ipw, family = binomial())
summary(fit)
```

We will elaborate on this workflow in Lab 5.

:::callout-note
Consider uncertainty of propensity score estimation!
$\rightarrow$ Use robust standard error or bootstrap confidence interval
:::